## GSMR

Note: Wasn't able to get the GSMR package working on Columbia cluster due to a <b>'Peer certificate cannot be authenticated with given CA certificates'<b> error.

### Data File

GSMR requires a data file that includes the effect size, std error, p value, and GWAS sample size for each phenotype. 

The original jazf1 summary stats files from Summer 2021 were

> **asthma** = /gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/asthma/Asthma_casesbyICD10codesANDselfreport_controlsbyselfreportandicd10_noautoimmuneincontrols_forbolt030720_ASTHMA.fastGWA.snp_stats.gz

> **t2d** = /gpfs/gibbs/pi/dewan/data/UKBiobank/results/FastGWA_results/results_imputed_data/T2D/diabetes_casesbyICD10andselfreport_controlswithoutautoiummune_030720_T2D.fastGWA.snp_stats.gz

> **waist** = /gpfs/gibbs/pi/dewan/data/UKBiobank/results/BOLTLMM_results/results_imputed_data/INT-WAIST/UKB_caucasians_BMIwaisthip_AsthmaAndT2D_INT-WAIST_withagesex_042020_rankNorm_WAIST.boltlmm.snp_stats.gz

But since these files don't have the sample size, used Yining's rerun regenie files for jazf1 to create the datafile for GSMR

Yining's regenie sumstats files are located on the Yale cluster at:

> **asthma**:/home/yl2385/work/21fall/220110_regenie_asthma_rerun/asthma_PC10_step2_imp.regenie_ASTHMA.regenie

> **t2d**:/home/yl2385/work/21fall/220110_regenie_T2D_rerun/T2D_PC10_step2_imp.regenie_T2D.regenie

> **waist**:/home/yl2385/work/21fall/220110_regenie_WC_rerun/WC_PC10_step2_imp.regenie_WAISTcirc_invranknorm.regenie

Moved them to Columbia's cluster and they're currently under:

> **asthma**:/mnt/mfs/hgrcgrid/homes/tf2478/20220302_regenie_rerun_yining/asthma_PC10_step2_imp.regenie_ASTHMA.regenie

> **t2d**:/mnt/mfs/hgrcgrid/homes/tf2478/20220302_regenie_rerun_yining/T2D_PC10_step2_imp.regenie_T2D.regenie

> **waist**:/mnt/mfs/hgrcgrid/homes/tf2478/20220302_regenie_rerun_yining/WC_PC10_step2_imp.regenie_WAISTcirc_invranknorm.regenie

Steps for creating the datafile located in **workflow_gsmr_data_031622.ipynb**. This notebook was run on Columbia's cluster

### LD File

GSMR example format uses gcta64 to create the LD file for snps, but chose to use [Region Extraction](https://github.com/cumc/bioworkflows/blob/master/GWAS/Region_Extraction.ipynb) for LD files instead

The bfiles used for the regenie rerun
> /mnt/mfs/statgen/archive/UKBiobank_Yale_transfer/pleiotropy_geneticfiles/UKB_Caucasians_phenotypeindepqc120319_updated082020removedwithdrawnindiv

However in this bfile there are only 96 snps in the jazf1 region, and only 74 of these snps are snps in the asthma regenie file. (Did not test for t2d or waist circumference but 96 snps are still less than the full number of snps for all three phenotypes).

Note: The rsid on the bfile do not match the rsid on the regenie. Used `chr:pos_a1_a2` to identify intersections for snps between the two files

The bgen files used for the regenie rerun. Regenie was run with all chr1..22 but jazf1 is only in chr 7
> /mnt/mfs/statgen/archive/UKBiobank_Yale_transfer/ukb39554_imputeddataset/ukb_imp_chr7_v3.bgen

**TODO**: The bgen files are in HG19, but the regenie are in HG38. Need to [liftover](https://github.com/cumc/bioworkflows/blob/master/GWAS/liftover.ipynb) the regenie to HG19 to see if all the regenie snps are in the bgen file prior to extraction to determine LD (skipped this to get Region Extraction working)

#### Region Extraction

There were problems with the region extraction so wasn't able to run it through, but the pipeline is fixed at this point in time.

The **phenotype** files used to rerun the regenie were in Yale's cluster in the same directories as the regenie files. They were pulled to Columbia's cluster. Located in

> **asthma**: /mnt/mfs/hgrcgrid/homes/tf2478/20220302_regenie_rerun_yining/pheno_asthma_ind_PC.txt

> **t2d**: /mnt/mfs/hgrcgrid/homes/tf2478/20220302_regenie_rerun_yining/pheno_asthma_ind_PC.txt

> **waist**: /mnt/mfs/hgrcgrid/homes/tf2478/20220302_regenie_rerun_yining/pheno_WC_ind_PC.txt

**TODO**: These are the full phenotype files. Need to subset the columns prior to running through Region Extraction. 

Format for files `FID IID sex [pheno] age PC1 PC2`

Created the **unrelated samples** file with header at
> /mnt/mfs/statgen/archive/UKBiobank_Yale_transfer/pleiotropy_geneticfiles/unrelated_n307259/UKB_genotypedatadownloaded0830_unrelatedCAUConly_307259indiv_529024snps_120319_withdrawnsubjects021820.fam_IDonly_wheader

The **geno** files used is the bgen file previously mentioned

The **sumstats** file is the regenie file for each phenotype. Note that since bgen is in HG19 and regenie are in HG38, need to use files produced from liftover as sumstats input for region extraction

The **region file** is
> /mnt/mfs/hgrcgrid/homes/tf2478/20220302_regenie_rerun_yining/jazf1.clumped_region

The **format config is**
> /home/dmc2245/project/UKBB_GWAS_dev/data/regenie_template.yml

**TODO**: Run region extraction to obtain the LD files necessary for analysis

### Analysis

Used the **gsmr_031622.ipynb** file on personal machine since GSMR would not download on Columbia's cluster. Successfully ran the example scripts. Was testing on Asthma-T2D, but dealt with issues regarding the LD.